In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('coffee_beans.csv')
df

,class index,filepaths,labels,data set
0,0,train/Dark/dark (1).png,Dark,train
1,0,train/Dark/dark (10).png,Dark,train
2,0,train/Dark/dark (100).png,Dark,train
3,0,train/Dark/dark (101).png,Dark,train
4,0,train/Dark/dark (102).png,Dark,train
...,...,...,...,...
1595,3,test/Medium/medium (95).png,Medium,test
1596,3,test/Medium/medium (96).png,Medium,test
1597,3,test/Medium/medium (97).png,Medium,test
1598,3,test/Medium/medium (98).png,Medium,test


In [12]:
image_path = df['filepaths']
image_path.iloc[0]

'train/Dark/dark (1).png'

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

# 2. Prepare Image Data Generator
# -----------------------------
# We'll rescale pixel values to [0,1]
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Train generator
train_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,  # filepaths in CSV are already relative/full paths
    x_col='filepaths',
    y_col='labels',
    subset='training',
    target_size=(64, 64),   # resize images
    batch_size=32,
    class_mode='categorical'  # multi-class classification
)

# Validation generator
val_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,
    x_col='filepaths',
    y_col='labels',
    subset='validation',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

# -----------------------------
# 3. Build Single-Layer Model
# -----------------------------
num_classes = train_gen.num_classes

model = Sequential([
    Flatten(input_shape=(64, 64, 3)),  # flatten 64x64 RGB image
    Dense(num_classes, activation='softmax')  # single output layer
])

# -----------------------------
# 4. Compile Model
# -----------------------------
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# -----------------------------
# 5. Train Model
# -----------------------------
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

# -----------------------------
# 6. Save Model
# -----------------------------
model.save('single_layer_model.h5')


Found 1280 validated image filenames belonging to 4 classes.
Found 320 validated image filenames belonging to 4 classes.


AttributeError: 'DataFrameIterator' object has no attribute 'num_classes'